<a href="https://colab.research.google.com/github/rameshbabulakshmanan84/NLP-series/blob/master/Aug_8_Clinicalbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING']='1'

In [ ]:
import pandas as pd
import csv
path='/content/drive/My Drive/NLP/ClinicalBiobert/Clinicalbert-10000/BC2GM-IOB-Train_10000.xlsx'
data=pd.read_excel(path,nrows=5012,delimiter='\t',names=["Words","tags"],encoding='unicode_escape')

In [ ]:
data.tail(50)

,Words,tags
4962,expression,O
4963,of,O
4964,SHP,B-GENE
4965,-,I-GENE
4966,1,I-GENE
4967,resulted,O
4968,in,O
4969,a,O
4970,approximately,O
4971,25,O


In [ ]:
data.shape

(5012, 2)

In [ ]:
data['tags'].isnull().sum()

1

In [ ]:
data['tags']=data['tags'].fillna('O')

In [ ]:
data['tags'].isnull().sum()
data['tags'].unique()

array(['O', 'B-GENE', 'I-GENE', 'B-DISEASE', 'I-DISEASE', 'B-DRUGS',
       'I-DRUGS'], dtype=object)

In [ ]:
data['tags'].value_counts()

O            4271
I-GENE        246
B-GENE        187
B-DRUGS        87
I-DRUGS        85
I-DISEASE      79
B-DISEASE      57
Name: tags, dtype: int64

Concatenate the words into a sentence - Make sure we convert by using str(input)

In [ ]:
sentences1=[]
tags1=[]
words=data['Words'].values.tolist()
tag=data['tags'].values.tolist()
#words
temp_sen=[]
temp_tag=[]
count=0
for i in words:
  #print(i)
  
  if i != 'End':
    temp_sen.append(str(i))
    temp_tag.append(str(tag[count]))
    count+=1
  else:
    sentences1.append(temp_sen)
    tags1.append(temp_tag)
    temp_sen=[]
    temp_tag=[]

In [ ]:
 #check the data type pf the sentences
 type(sentences1[0][2])

str

In [ ]:
#Print the length of tags1 and sentences1
print(len(tags1))
print(len(sentences1))

169
169


In [ ]:
#detect the maximum and mimim length of the sentences
l=[]
for i in sentences1:
  l.append(len(i))
print(max(l))
print(min(l))


105
2


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

In [ ]:
#map the tags with integers -> tags refer to ner
#data["tag"]:retrieves unique tags, set(data["tag"]) -> refers to set of distinct values
tag_values=list(set(data["tags"].values))
tag_values.append("PAD")
#create a dictionary of tag and values
tag2idx={t:i for i,t in enumerate(tag_values)}

In [ ]:
print(tag2idx)
#Tags available :
# I-GENE,B-GENE

{'I-DISEASE': 0, 'I-DRUGS': 1, 'B-GENE': 2, 'I-GENE': 3, 'B-DRUGS': 4, 'B-DISEASE': 5, 'O': 6, 'PAD': 7}


Set up the transformers library

In [ ]:
!pip3 install transformers


     |████████████████████████████████| 778kB 4.8MB/s 
     |████████████████████████████████| 890kB 23.7MB/s 
     |████████████████████████████████| 3.0MB 31.8MB/s 
     |████████████████████████████████| 1.1MB 42.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=6a28486a7c3d796814b10f1aa72cb6ad09d27e166ca02447eb178ac5af07ae27
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
#use pytorch implementation of bert
import torch
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler
from transformers import AutoTokenizer,AutoModelForTokenClassification,AutoModel
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
biobert_tokenizer=AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")

INFO:filelock:Lock 139867528904656 acquired on /root/.cache/torch/transformers/f8aa55d48600e63a39633cbb1bd933404d161174c7cabf1b4059b574404d6c19.08d27a67caf57f599c177c8e7fefc24d3e2f9a4e6ed2feca12f613f48c66ce07.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/dmis-lab/biobert-v1.1/config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpufb0nyqt


INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/dmis-lab/biobert-v1.1/config.json in cache at /root/.cache/torch/transformers/f8aa55d48600e63a39633cbb1bd933404d161174c7cabf1b4059b574404d6c19.08d27a67caf57f599c177c8e7fefc24d3e2f9a4e6ed2feca12f613f48c66ce07
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/f8aa55d48600e63a39633cbb1bd933404d161174c7cabf1b4059b574404d6c19.08d27a67caf57f599c177c8e7fefc24d3e2f9a4e6ed2feca12f613f48c66ce07
INFO:filelock:Lock 139867528904656 released on /root/.cache/torch/transformers/f8aa55d48600e63a39633cbb1bd933404d161174c7cabf1b4059b574404d6c19.08d27a67caf57f599c177c8e7fefc24d3e2f9a4e6ed2feca12f613f48c66ce07.lock
INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/dmis-lab/biobert-v1.1/config.json from cache at /root/.cache/torch/transformers/f8aa55d48600e63a39633cbb1bd933404d161174c7cabf1b4059b574404d6c19.08d27a67c

INFO:filelock:Lock 139870113279784 acquired on /root/.cache/torch/transformers/788ec7f94fa5412d56872a6c92079f8f48fb3dfbb1994fdeb251302999562180.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/dmis-lab/biobert-v1.1/vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmp6oz3jqso


INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/dmis-lab/biobert-v1.1/vocab.txt in cache at /root/.cache/torch/transformers/788ec7f94fa5412d56872a6c92079f8f48fb3dfbb1994fdeb251302999562180.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/788ec7f94fa5412d56872a6c92079f8f48fb3dfbb1994fdeb251302999562180.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:filelock:Lock 139870113279784 released on /root/.cache/torch/transformers/788ec7f94fa5412d56872a6c92079f8f48fb3dfbb1994fdeb251302999562180.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1.lock


INFO:filelock:Lock 139870113279784 acquired on /root/.cache/torch/transformers/29bcdc1d6651ef6310f8743ffb91ebec1135819dde9448330d5c636dc614425f.275045728fbf41c11d3dae08b8742c054377e18d92cc7b72b6351152a99b64e4.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/dmis-lab/biobert-v1.1/special_tokens_map.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpd104zebr


INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/dmis-lab/biobert-v1.1/special_tokens_map.json in cache at /root/.cache/torch/transformers/29bcdc1d6651ef6310f8743ffb91ebec1135819dde9448330d5c636dc614425f.275045728fbf41c11d3dae08b8742c054377e18d92cc7b72b6351152a99b64e4
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/29bcdc1d6651ef6310f8743ffb91ebec1135819dde9448330d5c636dc614425f.275045728fbf41c11d3dae08b8742c054377e18d92cc7b72b6351152a99b64e4
INFO:filelock:Lock 139870113279784 released on /root/.cache/torch/transformers/29bcdc1d6651ef6310f8743ffb91ebec1135819dde9448330d5c636dc614425f.275045728fbf41c11d3dae08b8742c054377e18d92cc7b72b6351152a99b64e4.lock


INFO:filelock:Lock 139867542566952 acquired on /root/.cache/torch/transformers/026b114b2c81e958018d6178d7a172b0ecc5db56f6f8923f0679696c4e09a0ff.33188cc140c92703fe59093b4841dbac6ef481e1a8ab71edfbef8649b9c44029.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/dmis-lab/biobert-v1.1/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpuyd_t_zk


INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/dmis-lab/biobert-v1.1/tokenizer_config.json in cache at /root/.cache/torch/transformers/026b114b2c81e958018d6178d7a172b0ecc5db56f6f8923f0679696c4e09a0ff.33188cc140c92703fe59093b4841dbac6ef481e1a8ab71edfbef8649b9c44029
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/026b114b2c81e958018d6178d7a172b0ecc5db56f6f8923f0679696c4e09a0ff.33188cc140c92703fe59093b4841dbac6ef481e1a8ab71edfbef8649b9c44029
INFO:filelock:Lock 139867542566952 released on /root/.cache/torch/transformers/026b114b2c81e958018d6178d7a172b0ecc5db56f6f8923f0679696c4e09a0ff.33188cc140c92703fe59093b4841dbac6ef481e1a8ab71edfbef8649b9c44029.lock


INFO:transformers.tokenization_utils_base:loading file https://s3.amazonaws.com/models.huggingface.co/bert/dmis-lab/biobert-v1.1/vocab.txt from cache at /root/.cache/torch/transformers/788ec7f94fa5412d56872a6c92079f8f48fb3dfbb1994fdeb251302999562180.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1
INFO:transformers.tokenization_utils_base:loading file https://s3.amazonaws.com/models.huggingface.co/bert/dmis-lab/biobert-v1.1/added_tokens.json from cache at None
INFO:transformers.tokenization_utils_base:loading file https://s3.amazonaws.com/models.huggingface.co/bert/dmis-lab/biobert-v1.1/special_tokens_map.json from cache at /root/.cache/torch/transformers/29bcdc1d6651ef6310f8743ffb91ebec1135819dde9448330d5c636dc614425f.275045728fbf41c11d3dae08b8742c054377e18d92cc7b72b6351152a99b64e4
INFO:transformers.tokenization_utils_base:loading file https://s3.amazonaws.com/models.huggingface.co/bert/dmis-lab/biobert-v1.1/tokenizer_config.json from cache at /root/.cache/torch/transf

In [ ]:
#import clinicalbiobert
#scibert_tokenizer=AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
#clibert_model=AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/emilyalsentzer/Bio_ClinicalBERT/config.json from cache at /root/.cache/torch/transformers/f1b766d4011bf16cfd52c7be6150a4f5d4c2256eaca3c2edb073f468005f08a2.e99f38979b92a72e708709cb38397fa864caf99dd2e2ed75826560450aa14a0c
INFO:transformers.configuration_utils:Model config BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 28996
}

INFO:transformers.tokenization_utils_base:Model name 'emilyalsentzer/Bio_ClinicalBERT' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
#Set the max length and batchsize
MAX_LEN = 64
Batch_Size = 32

In [ ]:
#device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
#n_gpu=torch.cuda.device_count()

In [ ]:
def token_and_preserve_labels(sentence,text_labels):
  tokenized_sentence=[]
  labels=[]

  for word,label in zip(sentence,text_labels):
    #tokenize the word and count of # of subwords that word is toeknized into 
    #demonstrators ['demons', '##tra', '##tors']
    tokenized_word=biobert_tokenizer.tokenize(word)
    n_subwords=len(tokenized_word)

    # Add the tokenized word to the final tokenized word list
    tokenized_sentence.extend(tokenized_word)

    # Add the same label to the new list of labels `n_subwords` times
    labels.extend([label] * n_subwords)

  return tokenized_sentence,labels 

In [ ]:
#sentence and corresponsing labels 
#sentence form : [list of words]
#labels form : #labels form [ list of ner tags]
tokenized_texts_and_labels=[token_and_preserve_labels(sent,labels) for sent,labels in zip(sentences1,tags1)]

In [ ]:
print(tokenized_texts_and_labels)

[(['I', '##mm', '##uno', '##his', '##to', '##chemical', 'stain', '##ing', 'was', 'positive', 'for', 'S', '-', '100', 'in', 'all', '9', 'cases', 'stained', ',', 'positive', 'for', 'HM', '##B', '-', '45', 'in', '9', '(', '90', '%', ')', 'of', '10', ',', 'and', 'negative', 'for', 'c', '##yt', '##oker', '##ati', '##n', 'in', 'all', '9', 'cases', 'in', 'which', 'my', '##x', '##oid', 'me', '##lan', '##oma', 'remained', 'in', 'the', 'block', 'after', 'previous', 'sections', '.'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'I-GENE', 'I-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'B-GENE', 'I-GENE', 'I-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'B-GENE', 'B-GENE', 'B-GENE', 'B-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DISEASE', 'B-DISEASE', 'B-DISEASE', 'I-DISEASE', 'I-DISEASE', 'I-DISEASE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['Ch', '##lora', '##mp', '##hen', '##ico', '##l', 'ace', '##ty', '##lt', '##ran', '##s', '##fer

In [ ]:
# split text and labels
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]

labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

In [ ]:
print(tokenized_texts[4])

['C', '##lon', '##ing', 'and', 'se', '##quencing', 'of', 'the', 'upstream', 'region', 'of', 'p', '##ep', '##X', 'revealed', 'the', 'presence', 'of', 'two', 'OR', '##F', '##s', 'of', '360', 'and', '1', ',', '33', '##8', 'b', '##p', 'that', 'were', 'shown', 'to', 'be', 'able', 'to', 'en', '##code', 'proteins', 'with', 'high', 'ho', '##mology', 'to', 'G', '##ln', '##R', 'and', 'G', '##ln', '##A', 'proteins', ',', 'respectively', '.']


In [ ]:
print(labels[4])

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-GENE', 'O', 'B-GENE']


In [ ]:
input_ids = pad_sequences([biobert_tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],maxlen=MAX_LEN,dtype="long", value=0.0,
                          truncating="post", padding="post")

In [ ]:
print(len(input_ids))

169


In [ ]:
#tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
#                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
#                     dtype="long", truncating="post")

In [ ]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],value=tag2idx["PAD"],maxlen=MAX_LEN,padding="post",dtype="long", truncating="post")

In [ ]:
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

In [ ]:
print(attention_masks)

[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.

In [ ]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids,tags,random_state=2018,test_size=0.4)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,random_state=2018, test_size=0.4)

In [ ]:
#convert all tensor format
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [ ]:
tr_tags.shape

torch.Size([101, 64])

In [ ]:
val_tags.shape

torch.Size([68, 64])

In [ ]:
tr_inputs.shape

torch.Size([101, 64])

In [ ]:
#train_data in the form - input tokens, attention masks for the corresponding tokens and the tags for the tokens
train_data = TensorDataset(tr_inputs,tr_masks,tr_tags)
#returns random indices from the data source
train_sampler = RandomSampler(train_data)
#dataloader combines the dataset and the sampler 
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=Batch_Size)
valid_data = TensorDataset(val_inputs, val_masks, val_tags)
#samples elements sequentially in the same order
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data,sampler=valid_sampler, batch_size=Batch_Size)

In [ ]:
for input,mask,tag in train_dataloader:
  print(mask.shape)
  print(input.shape)
  print(tag.shape)

torch.Size([32, 191])
torch.Size([32, 191])
torch.Size([32, 191])
torch.Size([32, 191])
torch.Size([32, 191])
torch.Size([32, 191])
torch.Size([32, 191])
torch.Size([32, 191])
torch.Size([32, 191])
torch.Size([32, 191])
torch.Size([32, 191])
torch.Size([32, 191])
torch.Size([32, 191])
torch.Size([32, 191])
torch.Size([32, 191])


Setup the Bert model for finetuning

In [ ]:
import transformers
#bertfortokenclassification addsa classifier on top of bert model
from transformers import BertForTokenClassification,AdamW,AutoConfig
#from transformers import 

In [ ]:
#Use small learning rates combined with bias correction to avoid vanishing gradients early in training.
#Increase the number of iterations considerably and train to (almost) zero training loss while
#making use of early stopping.

In [ ]:
#model1 = BertForTokenClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT",num_labels=len(tag2idx), output_attentions = False,output_hidden_states = False)

In [ ]:
model1=BertForTokenClassification.from_pretrained("dmis-lab/biobert-v1.1",num_labels=len(tag2idx), output_attentions = False,output_hidden_states = False)

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/dmis-lab/biobert-v1.1/config.json from cache at /root/.cache/torch/transformers/f8aa55d48600e63a39633cbb1bd933404d161174c7cabf1b4059b574404d6c19.08d27a67caf57f599c177c8e7fefc24d3e2f9a4e6ed2feca12f613f48c66ce07
INFO:transformers.configuration_utils:Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7
  },
  "layer_norm_e

INFO:transformers.file_utils:storing https://cdn.huggingface.co/dmis-lab/biobert-v1.1/pytorch_model.bin in cache at /root/.cache/torch/transformers/a5ea36102a9f4213c56a03820049979ab9dfc8c2c8e63c20095e69050c8b3e2c.1a16d1ac9f74d161ec6db0dad3d9eebd95872ad73fc70c8fc92bffc89cf0b84c
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/a5ea36102a9f4213c56a03820049979ab9dfc8c2c8e63c20095e69050c8b3e2c.1a16d1ac9f74d161ec6db0dad3d9eebd95872ad73fc70c8fc92bffc89cf0b84c
INFO:filelock:Lock 139867489736856 released on /root/.cache/torch/transformers/a5ea36102a9f4213c56a03820049979ab9dfc8c2c8e63c20095e69050c8b3e2c.1a16d1ac9f74d161ec6db0dad3d9eebd95872ad73fc70c8fc92bffc89cf0b84c.lock
INFO:transformers.modeling_utils:loading weights file https://cdn.huggingface.co/dmis-lab/biobert-v1.1/pytorch_model.bin from cache at /root/.cache/torch/transformers/a5ea36102a9f4213c56a03820049979ab9dfc8c2c8e63c20095e69050c8b3e2c.1a16d1ac9f74d161ec6db0dad3d9eebd95872ad73fc70c8fc92bffc89c

INFO:transformers.modeling_utils:All model checkpoint weights were used when initializing BertForTokenClassification.

You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model2=BertForTokenClassification.from_pretrained("dmis-lab/biobert-v1.1",num_labels=len(tag2idx), output_attentions = False,output_hidden_states = False)

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/dmis-lab/biobert-v1.1/config.json from cache at /root/.cache/torch/transformers/f8aa55d48600e63a39633cbb1bd933404d161174c7cabf1b4059b574404d6c19.08d27a67caf57f599c177c8e7fefc24d3e2f9a4e6ed2feca12f613f48c66ce07
INFO:transformers.configuration_utils:Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7
  },
  "layer_norm_e

In [ ]:
#model2=BertForTokenClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT",num_labels=len(tag2idx), output_attentions = False,output_hidden_states = False)

In [ ]:
import torch
#specify GPU as device
if torch.cuda.is_available():
  device=torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
model1.num_labels

8

In [ ]:
# Tell pytorch to run this model on the GPU.
model1.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
#Print the different parameters
params=list(model1.named_parameters())

#print
print("the model has {:} number of parameters".format(len(params)))

the model has 201 number of parameters


In [ ]:
for p in params[0:5]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print("---First Transformer---")
for p in params[5:21]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
for p in params[190:200]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))
print("---output layer---")
for p in params[-4:]:
  print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

bert.embeddings.word_embeddings.weight                  (28996, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)
---First Transformer---
bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (768,)
bert.encoder.layer.0.attention.output.LayerNorm.weight        (768,)
bert.encod

In [ ]:
#grab the optimzer
optimizer=AdamW(model1.parameters(),#ξ=0.95 and lr=2.0e-5.
               lr=2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5 
               eps=1e-8) #a very small number to prevent any division by zero in the implementation)

In [ ]:
model1.config

BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 28996
}

In [ ]:
from transformers import get_linear_schedule_with_warmup

In [ ]:
epochs=2

In [ ]:
total_steps = len(train_dataloader) * epochs

In [ ]:
#Create a schedule with a learning rate that decreases linearly from the initial lr set in the optimizer to 0, after a warmup period during which it increases linearly from 0 to the initial lr set in the optimizer.

scheduler=get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,#deafault value
    num_training_steps=total_steps)


In [ ]:
from sklearn.metrics import f1_score, accuracy_score,classification_report,confusion_matrix

In [ ]:
import random
import numpy as np

In [ ]:
# Set the seed value all over the place to make this reproducible.
seed_val=42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
#for ech epoch
#Run the training for the given epochs - typically between 2 and 4
#range 0 to 3

loss_values, validation_loss_values = [], []


for _ in trange(epochs,desc="EPOCH"):
  print("")
  #print ---Epoch 1/2 ----
  #print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
  print('Training...')

  #Reset the total loss for this epoch
  total_train_loss=0
  
  #train the model
  model1.train()

  for step,batch in enumerate(train_dataloader):
    print(step)
    #print(batch)

    #add the batch to the GPU
    batch=tuple(t.to(device) for t in batch)
    #print(batch.shape)
    b_input_ids,b_input_mask,b_labels=batch
    #print(b_labels)
  #always clear the gradients before doing a backward pass
    model1.zero_grad()

   #perform a forward pass
    loss,logits=model1(b_input_ids,token_type_ids=None,attention_mask=b_input_mask,labels=b_labels)
    
    #logits = logits.detach().cpu().numpy()
    #print("step number -", step)
    #print("score--",logits.shape)
    #outout shape in the form : 32*75*18 --32 batch size,75 --> sequence size, 18 --> Tags 
    #print("argmax--")
    #print("logits--",logits)
    #print("argmax--",np.argmax(logits,axis=1))
   #calculate the accumulating loss
   #`loss` is a Tensor containing a # single value; the `.item()` function just returns the Python value 
    total_train_loss+=loss.item()

   # Perform a backward pass to calculate the gradients.
   #compute gradient of loss w.r.t all the parameters in loss that have requires_grad = True and store them in parameter.grad attribute for every parameter.
    loss.backward()

   # Clip the norm of the gradients to 1.0.
   # This is to help prevent the "exploding gradients" problem
   #within a specific range (clip)
    torch.nn.utils.clip_grad_norm_(model1.parameters(), 1.0)

     
   # Update parameters and take a step using the computed gradient.
   # The optimizer dictates the "update rule"--how the parameters are
   # modified based on their gradients, the learning rate, etc.
   #updates all the parameters based on parameter.grad
    optimizer.step()

   # Update the learning rate.
    scheduler.step()
#Return to the epoch
   # Calculate the average loss over all of the batches.  
  avg_train_loss= total_train_loss/len(train_dataloader)
  print("Average train loss: {}".format(avg_train_loss))
  #append the loss for trainig
  loss_values.append(avg_train_loss)
  print(" Training is completed")
  print("Running Validation... ")

  #put the model in evaluation mode
  model1.eval()

  # Tracking variables 
  total_eval_accuracy = 0
  total_eval_loss = 0
  nb_eval_steps = 0
  predictions , true_labels = [], []

  for batch in valid_dataloader:

    #add the batch to the GPU
    batch=tuple(t.to(device) for t in batch)
    b_input_ids,b_input_mask,b_labels=batch
   
    #print("validation labels:",b_labels)
     #always clear the gradients
     #in evaluation mode, you can't do backprog, to save time, dont compute the gradients
    with torch.no_grad():
      #perform a forward pass
      loss,logits=model1(b_input_ids,token_type_ids=None,attention_mask=b_input_mask,labels=b_labels)

    logits=logits.detach().cpu().numpy()
    label_ids=b_labels.to('cpu').numpy()
    label_ids=b_labels


   #calculate the accumulating loss
   #`loss` is a Tensor containing a # single value; the `.item()` function just returns the Python value 
    total_eval_loss+=loss.mean().item()
    predictions.extend([list(p) for p in np.argmax(logits,axis=2)])
    true_labels.extend(label_ids)

  eval_loss=total_eval_loss/len(valid_dataloader) 
  validation_loss_values.append(eval_loss)
  print("Validation loss: {}".format(eval_loss))
  #do predictions only for the non-padded tags
  pred_tags=[tag_values[p_i] for p,l in zip(predictions,true_labels) for p_i,l_i in zip(p,l) if tag_values[l_i] != "PAD"]

  valid_tags=[tag_values[l_i] for l in true_labels for l_i in l if tag_values[l_i]!="PAD"]

  print("Confusion matrix:{}".format(confusion_matrix(pred_tags, valid_tags)))
  print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
  print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags,average='weighted')))
  print("Classification Report : {}".format(classification_report(pred_tags,valid_tags)))

EPOCH:   0%|          | 0/2 [00:00<?, ?it/s]


Training...
0
1
2
3
Average train loss: 1.6947075426578522
 Training is completed
Running Validation... 


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
EPOCH:  50%|█████     | 1/2 [00:02<00:02,  2.24s/it]

Validation loss: 1.1322603225708008
Confusion matrix:[[   0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0]
 [  28   53   83   47   44   77 2264]]
Validation Accuracy: 0.8721109399075501
Validation F1-Score: 0.9316872427983538
Classification Report :               precision    recall  f1-score   support

   B-DISEASE       0.00      0.00      0.00         0
     B-DRUGS       0.00      0.00      0.00         0
      B-GENE       0.00      0.00      0.00         0
   I-DISEASE       0.00      0.00      0.00         0
     I-DRUGS       0.00      0.00      0.00         0
      I-GENE       0.00      0.00      0.00         0
           O       1.00      0.87      0.93      2596

    accuracy                           0.87      2596
   macro avg       0.14      0.12      0.13      2596
weighted avg       1.00      0.87 

EPOCH: 100%|██████████| 2/2 [00:04<00:00,  2.02s/it]

Validation loss: 0.8288291096687317
Confusion matrix:[[   0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0]
 [  28   53   83   47   44   77 2264]]
Validation Accuracy: 0.8721109399075501
Validation F1-Score: 0.9316872427983538
Classification Report :               precision    recall  f1-score   support

   B-DISEASE       0.00      0.00      0.00         0
     B-DRUGS       0.00      0.00      0.00         0
      B-GENE       0.00      0.00      0.00         0
   I-DISEASE       0.00      0.00      0.00         0
     I-DRUGS       0.00      0.00      0.00         0
      I-GENE       0.00      0.00      0.00         0
           O       1.00      0.87      0.93      2596

    accuracy                           0.87      2596
   macro avg       0.14      0.12      0.13      2596
weighted avg       1.00      0.87 

Demostrate the testing of sentence 

In [ ]:
test_sentence = "Cerebral125 albumin was increased to similar proportions in those groups submitted to hyperosmolality."

In [ ]:
#tokenize  the text
tokenized_sentence=scibert_tokenizer.encode(test_sentence)
#the output is sequence of ids (token to ids)

In [ ]:
print(tokenized_sentence)

[101, 21831, 11964, 1571, 1312, 1394, 1108, 2569, 1106, 1861, 21136, 1107, 1343, 2114, 7402, 1106, 177, 24312, 2155, 3702, 14258, 1785, 119, 102]


In [ ]:
#conver to tensor format
input_ids = torch.tensor([tokenized_sentence])

In [ ]:
# Tell pytorch to run this model on the GPU.
model1.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
#this 
input_ids=input_ids.cuda()


In [ ]:
 #batch=tuple(t.to(device) for t in batch)

In [ ]:
#run the sentence through the model
with torch.no_grad():
  logits=model1(input_ids)

In [ ]:
#run the sentence through the model
#with torch.no_grad():
#  logits_2=model2(input_ids)

In [ ]:
#label_indices_model2=torch.argmax(logits_2[0],axis=2)


In [ ]:
label_indices=torch.argmax(logits[0],axis=2)


In [ ]:
print(label_indices)

tensor([[2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2]],
       device='cuda:0')


In [ ]:
#print(label_indices_model2)

In [ ]:
#join the split tokens
tokens=scibert_tokenizer.convert_ids_to_tokens(input_ids[0])

In [ ]:
print(tokens)

['[CLS]', 'cerebral', '##12', '##5', 'album', '##in', 'was', 'increased', 'to', 'similar', 'proportions', 'in', 'those', 'groups', 'submitted', 'to', 'h', '##yper', '##os', '##mo', '##lal', '##ity', '.', '[SEP]']


In [ ]:
new_token,new_labels=[],[]
for token,label_idx in zip(tokens,label_indices[0]):
  if token.startswith("##"):
    #add at the end
    new_token[-1]=new_token[-1]+token[2:]
  else:
    #if its not special token
    new_labels.append(tag_values[label_idx])
    new_token.append(token) 

In [ ]:
for token,label in zip(new_token,new_labels):
  print("{}\t{}".format(token,label))
  

[CLS]	O
cerebral125	B-GENE
albumin	O
was	O
increased	O
to	O
similar	O
proportions	O
in	O
those	O
groups	O
submitted	O
to	B-GENE
hyperosmolality	O
.	O
[SEP]	O
